In [1]:
import gensim.downloader as api
#dataset = api.load("text8")
import math
import numpy as np
import h5py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  
import sys
sys.path.append("..")
import tensorflow as tf
from Vocabulary import *
import time
tf.keras.backend.clear_session()
import csv
import cloudpickle
from csv_writer import *

In [2]:
class lr_On_plato:
    lowest_loss = -1
    lowest_time = 0
    patience = 10
    factor = 0.005
    
    def notify_loss(self,loss,epoch):
        if(self.lowest_loss == -1):
            self.lowest_loss = loss
            self.lowest_time = epoch
        if(loss < self.lowest_loss):
            self.lowest_loss = loss
            self.lowest_time = epoch
        if(loss > self.lowest_loss and self.lowest_time + 10 < epoch):
            self.lowest_loss = loss
            self.lowest_time = epoch
            print("decreased LR")
            self.factor = self.factor * 0.5
    
    def get_lr(self,epoch):
        return self.factor
        

In [3]:
class ModelTrainer:
    def __init__(self,vocab_length,block_path,vector_size = 300):
        self.vector_size = vector_size
        self.tf_con_bias = None
        self.tf_bias = None
        self.weights = None
        self.tf_weights = None
        self.zeilen = 0
        self.spalten = 0
        # AND HERE IT IS AGAIN
        self.block_length = 20000
        self.amount_split = math.ceil(vocab_length/self.block_length)
        self.block_path = block_path
        self.vocab_length = vocab_length
    
    def prepare(self,filename):
        self.f = h5py.File('S:\\{filename}.hdf5'.format(filename=filename), "w")#plus experiment name
        #initalize all the HDF files
        self.con_weights = self.f.create_dataset("context-weights", (self.vocab_length, self.vector_size))
        self.weights = self.f.create_dataset("weights",(self.vector_size,self.vocab_length))
        self.context_bias = self.f.create_dataset("context-bias", (self.vocab_length,1))
        self.bias = self.f.create_dataset("bias", (1,self.vocab_length))

        self.init_matrices()
    
    def resume(self,filename):
        self.f = h5py.File('S:\\{filename}.hdf5'.format(filename=filename), "r+")#plus experiment name
        #initalize all the HDF files
        self.con_weights = self.f.get("context-weights")
        self.weights = self.f.get("weights")
        self.context_bias = self.f.get("context-bias")
        self.bias = self.f.get("bias")

    
    def init_matrices(self,chunk_size=10000):
        self.init_hdf_matrix(self.weights,-0.5,0.5,chunk_size)
        self.init_hdf_matrix(self.con_weights,-0.5,0.5,chunk_size)
        self.init_hdf_matrix(self.context_bias,-0.5,0.5,chunk_size)
        self.init_hdf_matrix(self.bias,-0.5,0.5,chunk_size)
    
    def init_hdf_matrix(self,hdf_data,min_value,max_value,block_length):
        if len(hdf_data) > len(hdf_data[0]):
            iterations = int(math.ceil(len(hdf_data) / block_length))
            for i in range(iterations):
                current_size = min(block_length,len(hdf_data)-block_length*i)
                hdf_data[i*block_length:(i+1)*block_length , :] = np.random.rand(current_size,len(hdf_data[0]))/self.vector_size
        else:
            iterations = int(math.ceil(len(hdf_data[0]) / block_length))
            for i in range(iterations):
                current_size = min(block_length,len(hdf_data[0])-block_length*i)
                hdf_data[:,i*block_length:(i+1)*block_length] = np.random.rand(len(hdf_data),current_size)/self.vector_size
            
    
    def load_blocks(self,zeilen,spalten):
        # load the hdf coocurence block
        if(zeilen >= spalten):
            template = "co_occurence_{i}_{j}.hdf5".format(i=zeilen,j=spalten)
        else:
            template = "co_occurence_{i}_{j}.hdf5".format(i=spalten,j=zeilen)
        file_path =  self.block_path + '\\' + template
        
        tmp_hf = h5py.File(file_path, "r+")
        coocurrence = tmp_hf.get("co-ocurrence")[:]
        if (spalten > zeilen):
            coocurrence = np.transpose(coocurrence)
        self.tf_co_occurences = tf.convert_to_tensor(coocurrence,dtype=tf.dtypes.float32)
        coocurrence = None
        tmp_hf.close()
        #Use normal matrix, if epsilon Shift, than add one to co-ocurence table to fix scaling and log
        self.tf_con_bias = tf.Variable(initial_value=self.context_bias[zeilen*self.block_length:(zeilen+1)*self.block_length,:],dtype=tf.dtypes.float32)
        self.tf_bias = tf.Variable(initial_value=self.bias[:,spalten*self.block_length:(spalten+1)*self.block_length],dtype=tf.dtypes.float32)
        self.tf_con_weights =  tf.Variable(initial_value=self.con_weights[zeilen*self.block_length:(zeilen+1)*self.block_length,:],dtype=tf.dtypes.float32)
        self.tf_weights = tf.Variable(initial_value=self.weights[:,spalten*self.block_length:(spalten+1)*self.block_length],dtype=tf.dtypes.float32)
    
    def save_blocks(self,zeilen,spalten):
        self.context_bias[zeilen*self.block_length:(zeilen+1)*self.block_length,:] = self.tf_con_bias.numpy()
        self.bias[0,spalten*self.block_length:(spalten+1)*self.block_length] = self.tf_bias.numpy()
        self.con_weights[zeilen*self.block_length:(zeilen+1)*self.block_length,:] = self.tf_con_weights.numpy()
        self.weights[:,spalten*self.block_length:(spalten+1)*self.block_length] = self.tf_weights.numpy()
    
    def _close_files(self):
        self.f.close()
    
    def train_splitted(self,epochs,optimizer_factory,lrOnPlato):
        csv_writer = CSV_writer("hallo")
        #These can stay in memory
        
        self.epsilon = tf.constant(1e-8) * tf.ones((self.block_length,self.block_length),dtype=tf.dtypes.float32)
        
        
        
        for epoch in range(0,epochs):
            cur_loss = 0
            for zeilen in range(0,self.amount_split):
                for spalten in range(0,self.amount_split):
                    # REMOVE THIS NEXT LINE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                    #if(zeilen < 7):
                    #    continue
                    self.zeilen = zeilen
                    self.spalten = spalten
                    
                    #load optimizer & blocks
                    if(epoch == 0):
                        optimizer = optimizer_factory.create()
                    else:
                        name = 'S://optimizer{z}-{s}'.format(z = zeilen,s = spalten)
                        with open(name, "rb") as file:
                            optimizer = cloudpickle.load(file)
                            #print('loaded optimizer')
                    optimizer.learning_rate.assign(lrOnPlato.get_lr(epoch))
                    self.load_blocks(zeilen,spalten)
                    print(zeilen,spalten)

                    #train
                    train = optimizer.minimize(self.loss, var_list=[self.tf_con_bias,self.tf_bias,
                                                              self.tf_con_weights,self.tf_weights])
                    cur_loss += self.loss()
                    print(train)
                    #print(optimizer.learning_rate)
                    
                    #save stuff
                    self.save_blocks(zeilen,spalten)
                    with open('S://optimizer{z}-{s}'.format(z = zeilen,s = spalten), 'wb') as file:
                        cloudpickle.dump(optimizer, file)
                        
                
            print('epoch'+str(epoch)+"loss:"+str(cur_loss.numpy()))
            lrOnPlato.notify_loss(cur_loss.numpy(),epoch)
            csv_writer.write(optimizer_factory.optimiser_name(),lrOnPlato.get_lr(epoch),epoch+1,cur_loss.numpy())
        self._close_files()
        return None

    def loss(self):
        ones_symetrical = tf.ones((self.block_length,self.block_length), dtype=tf.dtypes.float32, name=None)
    
        co_occurences = None
        #;ust the words context
        if(self.zeilen == self.amount_split - 1):
            difference = self.block_length - self.tf_con_bias.shape[0]
            add2_context_bias   = tf.zeros((difference,1),dtype=tf.dtypes.float32)
            add2_context_weights = tf.zeros((difference,self.vector_size),dtype=tf.dtypes.float32)
            
            context_weights       = tf.concat([self.tf_con_weights,add2_context_weights],axis = 0)
            con_bias_mat   = tf.concat([self.tf_con_bias,add2_context_bias],axis = 0) * ones_symetrical
        else:
            con_bias_mat   = self.tf_con_bias * ones_symetrical
            context_weights       = self.tf_con_weights
        
        co_occurences = self.tf_co_occurences
        #;ust the words without context
        if(self.spalten == self.amount_split - 1):
            difference = self.block_length - self.tf_bias.shape[1]
            add2_bias = tf.zeros((1,difference),dtype=tf.dtypes.float32)
            add2_weights = tf.zeros((self.vector_size,difference),dtype=tf.dtypes.float32)
            
            weights = tf.concat([self.tf_weights,add2_weights],axis = 1)
            bias_mat = tf.concat([self.tf_bias,add2_bias],axis=1) * ones_symetrical
        else:

            weights     = self.tf_weights
            bias_mat = self.tf_bias * ones_symetrical
          
                                                          
        bias_terms = bias_mat + con_bias_mat
    
        weight_matrix = tf.matmul(context_weights,weights)
        log_X = tf.math.log(co_occurences + self.epsilon)
        inner_sum = bias_terms + weight_matrix - log_X
        squared_sum = tf.math.square(inner_sum)
        weighted_sum = self.cut_function2(co_occurences) * squared_sum
        reduced = tf.math.reduce_sum(weighted_sum)
        return reduced
    
    alpha = tf.constant(0.75,dtype=tf.dtypes.float32)
    XMAX = tf.constant(100.0,dtype=tf.dtypes.float32)
    def cut_function2(self,value):
        clipped = tf.clip_by_value(value, clip_value_min = 0.0, clip_value_max=100.0)
        return tf.pow(clipped / self.XMAX, self.alpha)

In [ ]:
vocab = Vocabulary()
vocab.load('..\\vocabs\\base_vocabulary')
size = vocab.get_size()

class AdamFactory:
    def __init__(self,lr = 0.05):
        self.lr = lr
    def create(self):
        return tf.keras.optimizers.Adam(self.lr)
    def optimiser_name(self):
        return "Adam"
    
    
tf.keras.backend.clear_session()
trainer = ModelTrainer(size,"S:\\base_coocurrence_hdf")
trainer.prepare("baseline_preSanityCheck")
trainer.train_splitted(1,AdamFactory(),lr_On_plato())

trainer2 = ModelTrainer(size,'S:\\base_coocurrence_hdf')
trainer2.resume("baseline_preSanityCheck")
trainer2.train_splitted(5,AdamFactory(),lr_On_plato())

0 0
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
0 1
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
0 2
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
0 3
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
0 4
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
0 5
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
0 6
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
0 7
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
1 0
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
1 1
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
1 2
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
1 3
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
1 4
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
1 5
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
1 6
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
1 7
<tf.Variable 'UnreadV

7 6
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
7 7
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
epoch0loss:242080400.0
0 0
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
0 1
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
0 2
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
0 3
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
0 4
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
0 5
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
0 6
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
0 7
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
1 0
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
1 1
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
1 2
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
1 3
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
1 4
<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>
1 

In [ ]:
trainer2.resume("baseline_preSanityCheck")
trainer2.train_splitted(10,AdamFactory(0.001),lr_On_plato())